In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map

In [2]:
sitename = "nzd0161"
transect_id = "nzd0161-0135"
veg = pd.read_csv(f"Data/{sitename}/transect_time_series.csv")
veg.index = pd.to_datetime(veg.dates + "+00:00")
veg = veg[transect_id]
veg

dates
1999-08-26 22:04:44.971000+00:00    400.79
1999-11-14 22:04:42.286000+00:00    536.35
2000-01-01 22:04:37.439000+00:00    541.20
2000-02-18 22:04:29.218000+00:00       NaN
2000-05-24 22:03:46.662000+00:00    501.99
                                     ...  
2025-03-02 22:11:40.284000+00:00    495.17
2025-03-02 22:12:04.243000+00:00       NaN
2025-03-10 22:11:52.276000+00:00    513.37
2025-04-11 22:11:41.675000+00:00    521.50
2025-04-11 22:12:05.634000+00:00    255.98
Name: nzd0161-0135, Length: 370, dtype: float64

In [3]:
water = pd.read_csv(f"../CoastSat/data/{sitename}/transect_time_series_tidally_corrected.csv")
water.index = pd.to_datetime(water.dates)
water = water[transect_id]
water

dates
1999-08-26 22:04:44+00:00    395.395714
1999-09-27 22:04:48+00:00    403.748571
1999-11-14 22:05:06+00:00    411.821429
1999-12-25 21:58:52+00:00           NaN
2000-05-24 22:03:46+00:00    389.157143
                                ...    
2025-06-07 22:05:29+00:00           NaN
2025-06-15 22:05:40+00:00    406.380000
2025-08-01 22:11:42+00:00           NaN
2025-08-02 22:05:57+00:00    392.680000
2025-08-10 22:06:04+00:00    395.750000
Name: nzd0161-0135, Length: 373, dtype: float64

In [4]:
df = pd.DataFrame({"water": water, "veg": veg})
df

,water,veg
dates,,
1999-08-26 22:04:44+00:00,395.395714,NaN
1999-08-26 22:04:44.971000+00:00,NaN,400.79
1999-09-27 22:04:48+00:00,403.748571,NaN
1999-11-14 22:04:42.286000+00:00,NaN,536.35
1999-11-14 22:05:06+00:00,411.821429,NaN
...,...,...
2025-06-07 22:05:29+00:00,NaN,NaN
2025-06-15 22:05:40+00:00,406.380000,NaN
2025-08-01 22:11:42+00:00,NaN,NaN


In [5]:
px.scatter(df, trendline="ols", title=transect_id)

In [6]:
from coastsat import SDS_transects

def despike(chainage, threshold=40):
    chainage = chainage.dropna()
    if len(chainage) < 5:
        return chainage
    chainage, dates = SDS_transects.identify_outliers(
        chainage.tolist(), chainage.index.tolist(), threshold
    )
    return pd.Series(chainage, index=dates)

despiked = despike(veg)
smoothed = despiked.rolling("180d", min_periods=1).mean()
df = pd.DataFrame({"water": water, "raw": veg, "despiked": despiked, "smoothed": smoothed})
px.scatter(df, trendline="ols", title=transect_id)

In [7]:
shorelines = gpd.read_file("../CoastSat/shorelines.geojson")
shorelines = shorelines[shorelines.id.str.startswith("nzd")]
sites = sorted(shorelines.id.unique())

In [8]:
def process_site(sitename):
    try:
        df = pd.read_csv(f"Data/{sitename}/transect_time_series.csv")
    except:
        return
    df.dates = pd.to_datetime(df.dates)
    df.set_index("dates", inplace=True)

    satname = df.satname
    df = df.drop(columns="satname").apply(despike, axis=0)
    df["satname"] = satname

    for transect_id in df.drop(columns="satname").columns:
        df[transect_id] = df[transect_id].rolling("180d", min_periods=1).mean()

    df.reset_index(names="dates", inplace=True)
    df.to_csv(f"Data/{sitename}/transect_time_series_smoothed.csv", index=False)

process_map(
    process_site,
    shorelines.id.unique(),
    max_workers=32,
    chunksize=1,
    desc="Processing sites",
    position=0,
    leave=True,
)

Processing sites:   0%|          | 0/562 [00:00<?, ?it/s]

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

/tmp/ipykernel_1041540/4180380535.py:16: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



*********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

/tmp/ipykernel_1041540/4180380535.py:16: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



...

************************************************************************************************************************

/tmp/ipykernel_1041540/4180380535.py:16: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



********************************************************************

/tmp/ipykernel_1041540/4180380535.py:16: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,